In [8]:
from PIL import Image

In [9]:
import skimage.measure
def pooling_data(Iftrain, data, basepixel, pooling_size):
    
    #print pd.DataFrame(data)
     #压缩矩阵操作,取平均值，你可以自己写函数
    pooled_data = skimage.measure.block_reduce(data,pooling_size,  np.mean)
    #大于某个像素值 ==> 255
   
            
                              
    pooled_data = skimage.measure.block_reduce(data,pooling_size,  np.mean)
    if Iftrain == True:
        if basepixel > 0:
            pooled_data[pooled_data> basepixel] =  255
            pooled_data[pooled_data<= basepixel] = 0
    return pooled_data.T

In [10]:
import os
import numpy as np

def save_to_image(Iftrain, label_number, pixels,basepixel, i):
    MNIST = ""
    if Iftrain == True:
        MNIST = "TRIAN_DATA"
    else:
        MNIST = "TEST_DATA"
        
    image = Image.new('L',pixels.shape)
    columns = pixels.shape[0]
    rows = pixels.shape[1]
    
    for x in xrange(columns):
            for y in xrange(rows):
                    pix = pixels[y][x]
                    image.putpixel((x, y), int(pixels[y][x]))
    
    #print basepixel
    
    directory = MNIST+"/"+label_number +"/"+ str(columns)+"_"+str(rows)+"/"+"basepixel_"+str(basepixel)+"/"
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    image.save(directory+"/"+str(i) + '.png')

In [39]:
def save_to_one_numpy(Iftrain, train,  label, basepixel):
    MNIST = ""
    if Iftrain == False:
        MNIST = "TEST_DATA"
    else:
        return 
    size = list(train.shape)[1:]
    
    print size, len(train) ,"================================save_to_numpy==========================="
    
    directory = MNIST+"/"+ str(size[0])+"_"+str(size[1])+"/"+"basepixel_"+str(basepixel)+"/"

    #print pd.DataFrame(pixels)

    if not os.path.exists(directory):
        os.makedirs(directory)
    ###################################################
    #将数据转置后存入
    print "******************************************************"
    
    np.save(directory+'MNIST_train.npy', train)
    test = np.load(directory+'MNIST_train.npy')
    print np.all(train == test)  
    ###################################################
    np.save(directory+"MNIST_label.npy", label)

    label_test = np.load(directory+'MNIST_label.npy')
    #print np.all(label == label_test)  
    ##################################################

    with open(directory+'MNIST_train.txt','wb') as f:
        for row in train:
            
            np.savetxt(f,row,delimiter=',',fmt = "%03d", footer='\n')

    with open(directory+'MNIST_label.txt','wb') as f:
        for row in label:
            
            np.savetxt(f,row,delimiter=',',fmt = "%01d", footer='\n')            
    #x = np.loadtxt(directory+'MINST_train.txt',delimiter=',')
    #print np.all(x == train)

In [43]:
def save_to_numpy(Iftrain, train,  label, basepixel):
    MNIST = ""
    if Iftrain == True:
        MNIST = "TRIAN_DATA"
    else:
        MNIST = "TEST_DATA"
    
    size = list(train[0].shape)[1:]
    print size, len(train) ,"================================save_to_numpy==========================="
    for idx in range(len(train)):
        print "label_ ===========================",label[idx]
        directory = MNIST+"/"+str(idx)+"/" +"/"+ str(size[0])+"_"+str(size[1])+"/"+"basepixel_"+str(basepixel)+"/"

        #print pd.DataFrame(pixels)

        if not os.path.exists(directory):
            os.makedirs(directory)
        ###################################################
        #将数据转置后存入
        print "******************************************************"
        print train[idx]
        print label[idx]
        print "******************************************************"
        np.save(directory+'MNIST_train.npy', train[idx])
        test = np.load(directory+'MNIST_train.npy')
        print np.all(train[idx] == test)  
        ###################################################
        np.save(directory+"MNIST_label.npy", label[idx])
        
        label_test = np.load(directory+'MNIST_label.npy')
        #print np.all(label == label_test)  
        ##################################################

        with open(directory+'MNIST_train.txt','wb') as f:
            for row in train[idx]:
                 np.savetxt(f,row,delimiter=',',fmt = "%03d", footer='\n')

        with open(directory+'MNIST_label.txt','wb') as f:
            for row in label[idx]:
                 np.savetxt(f,row,delimiter=',',fmt = "%01d", footer='\n')            
        #x = np.loadtxt(directory+'MINST_train.txt',delimiter=',')
        #print np.all(x == train)

In [46]:
from PIL import Image
import struct
import numpy as np
import pandas as pd
import math

def deal_image(train_filename, label_filename, pooling_size_list, Debug_num = 0,Iftrain = True):
    
    ##############################################################################################
    train_file = open(train_filename, 'rb')
    buf = train_file.read()
    print "buf", len(buf)
    train_file.close()
    index = 0
    magic, images, rows, columns = struct.unpack_from('>IIII' , buf , index)
    print "magic", magic, "images", images, rows, columns
    index += struct.calcsize('>IIII')
    if Debug_num == 0:
        images_num = images
    else:
        images_num = Debug_num    
    ################################################################################################
    label_f = open(label_filename, 'rb')
    label_buf = label_f.read()
    label_index = 0
    struct.unpack_from('>II' , label_buf, label_index)
    label_index += struct.calcsize('>II')
    
    for size in pooling_size_list:
        for basepixel in [0]:
            
            pooled_size = (int(math.ceil(columns*1.0/size[0])) , int(math.ceil(rows*1.0/size[1])))
            #print pooled_size
            
            
            ###################################################################
            np_to_save = np.zeros((1,  pooled_size[0], pooled_size[1]))
            
            np_to_save_list = [np.zeros((1,  pooled_size[0], pooled_size[1])) for i in range(10)]
            
            
            #######
            test_save_all_one = np.zeros((images_num, pooled_size[0], pooled_size[1]))
            label_save_all_one = np.zeros((images_num, 1, 10))
            
                
                
            label_to_save = np.zeros((1, 1, 10))
            label_to_save_list = [np.zeros((1, 1, 10)) for i in range(10)]
            
            ###################################################################
            index = 0
            index += struct.calcsize('>IIII')
            ##############################
            ##############################
            label_index = 0
            label_index += struct.calcsize('>II')
            ##############################
            check = [False for i in range(10)]
            for i in xrange(images_num):
                # for debug
                

                normal_np = np.zeros((columns, rows))
                label_np = np.zeros(10)
                label_number = (struct.unpack_from('>B', label_buf, label_index)[0])
                
                label_index += struct.calcsize('>B')

                label_np[label_number] = 1
                #冗余的代码
                for x in xrange(columns):
                  for y in xrange(rows):
                    normal_np[y][x] = int(struct.unpack_from('>B', buf, index)[0])
                    index += struct.calcsize('>B')
                
                #normal_np 就是一个图片的矩阵 28*28
                #print normal_np.shape
                pooled_data = pooling_data(Iftrain, normal_np, basepixel,size)
              
                #print pooled_data.shape 
                pooled_cut_2 = pooled_data
                #print pooled_cut_2.shape, "pooled_shape_cut_2=================================================================="
                # 应该输出14*14
                
                
                test_save_all_one[i] = np.copy(pooled_cut_2)
                label_save_all_one[i] = np.copy(label_np)
                
                
                ###############################################################
                if check[label_number] == False:
                    #print "label_number_false================" ,label_number
                    #print "label_np_false=======================", label_np

                    np_to_save[0] = np.copy(pooled_cut_2)
                    
                    label_to_save[0] = np.copy(label_np)
                    
              
                                                            
            
                    np_to_save_list[label_number] =  np.copy(np_to_save)
                    #print "label_to_save", label_to_save
                    label_to_save_list[label_number] = np.copy(label_to_save)
                    #print "label_to_save_list_false", label_to_save_list
                    check[label_number] = True
                    
                else:
                    #print "label_number==================" ,label_number
                    
                    np_to_save[0] = np.copy(pooled_cut_2)
                    #print "label_np=======================", label_np
                    label_to_save[0] = np.copy(label_np)
                    
                    np_to_save_list[label_number]= np.row_stack((np_to_save_list[label_number], np.copy(np_to_save)))
                    label_to_save_list[label_number] = np.row_stack((label_to_save_list[label_number], np.copy(label_to_save)))
                #print "===============================np_to_save_list",np_to_save_list[label_number].shape, pooled_cut_2.shape

                ################################################################
                save_to_image(Iftrain, str(label_number), pooled_cut_2,basepixel, i)
            #print "label_to_save_list", label_to_save_list
            save_to_numpy(Iftrain, np_to_save_list, label_to_save_list, basepixel)
            save_to_one_numpy(Iftrain, test_save_all_one,label_save_all_one, basepixel)

In [47]:
#设置要池化的数据块大小
pooling_size_list = [(2,2),(1,1)]
#if Debug_num == 0, then save all images, Debug_num >0, save Debug_num images
#deal_image("train-images.idx3-ubyte", "train-labels.idx1-ubyte",pooling_size_list,Debug_num=10,Iftrain=True)
deal_image("t10k-images.idx3-ubyte","t10k-labels.idx1-ubyte", pooling_size_list,Debug_num=10,Iftrain=False)


buf 7840016
magic 2051 images 10000 28 28
[14, 14] 10 ================================save_to_numpy===========================
label_ =========================== [[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]]
******************************************************
[[[   0.      0.      0.      0.      0.      0.      0.      0.      0.
      0.      0.      0.      0.      0.  ]
  [   0.      0.      0.      0.      0.      0.      0.      0.      0.
      0.      0.      0.      0.      0.  ]
  [   0.      0.      0.      0.      0.      0.    112.25  239.75   34.5
      0.      0.      0.      0.      0.  ]
  [   0.      0.      0.      0.      0.     80.25  237.5   252.     96.25
     15.5     0.      0.      0.      0.  ]
  [   0.      0.      0.      0.     45.5   253.    252.    247.75  244.25
    194.25    0.      0.      0.      0.  ]
  [   0.      0.      0.      8.    191.5   252.    214.75   58.5    55.25
    245.75  115.25    1.25    0.      0.  ]
  [   0.      0.      0.    

[28, 28] 10 ================================save_to_numpy===========================
label_ =========================== [[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]]
******************************************************
[[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0

label_ =========================== [[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]

 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]]
******************************************************
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]

 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]]
******************************************************
True
[28, 28] 10 ================================save_to_numpy===========================
******************************************************
True


In [19]:
def read_to_numpy(basepixel, size):
    directory = "MINST_train/"+"basepixel_"+str(basepixel) +"/"+ str(size[0])+"_"+str(size[1])+"/"
    
    #print pd.DataFrame(pixels)
    print directory
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
    test = np.load(directory+'MINST_train.npy')
    
    img = Image.fromarray(test[1], 'L')
    img.show()
    print pd.DataFrame(test[1])

In [37]:
dic = {0:"0000000001", 1:"0000000010",2:"0000000100",3:0000001000,4:0000010000,5:0000100000,6:0001000000,7:0010000000,8:0100000000,9:1000000000}
dic[2]

64

In [70]:
a = np.array([[[1,3],[3,2]]])
a.shape

(1, 2, 2)

In [71]:
b = np.array([3,4])

b.shape

(2,)

In [74]:
np.concatenate(a, b)

TypeError: only integer scalar arrays can be converted to a scalar index

In [21]:
import numpy as np
from PIL import Image

# Creates a random image 100*100 pixels
mat = np.random.random((100,100))

# Creates PIL image
img = Image.fromarray(mat, 'L')
img.show() 

In [85]:
label_index = 0
label_index += struct.calcsize('>9B')
buf2 = open("train-labels.idx1-ubyte", "rb").read()
print struct.unpack_from('>B', buf2, 8)


(5,)


In [38]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [52]:
lst = [12,6,7,8,9,20]


In [54]:
lst.index(max(lst))

5

In [31]:
def rec_rate(count,label):
    global right_count
    for idx in range(len(count)):
        label_index = label[idx].index(1)
        count_max =max(count[idx])
        max_num = 0
        for item in count[idx]:
            if count_max == item:
                max_num =max_num +1
        if max_num ==1:
            if label_index ==count[idx].index(count_max):
                right_count =right_count +1
            else:
                right_count =right_count
    return right_count

count =[[0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0]]
label = [[0,0,0,0,0,1,0,0,0,0],
         [0,0,0,0,0,0,1,0,0,0],
         [0,0,0,0,0,0,1,0,0,0],
         [0,0,0,1,0,0,0,0,0,0]]
right_count = 0
right_count =rec_rate(count,label)
print " right_count", right_count

 right_count 0


In [ ]:
dict_para ={}
for i in range(vth_start,vth_end,vth_step):
    for j in range(exc_syn_stren_start,exc_syn_stren_end,exc_syn_stren_step):
        for k in range(inh_syn_stren _start,inh_syn_stren _end,inh_syn_stren _step):
            for m in range(A_pre_start,A_pre_end,A_pre_step):
                for n in range(A_post_start,A_post_end,A_post_step):
                    for h in range(leak_stren_start,leak_stren_end,leak_stren_step):
                        for z in range(bistable_stren_start,bistable_stren_end,bistable_stren_step):
                            keys ="vth is %d,exc_syn_stren is %d,inh_syn_stren is %d,A_pre is %d,A_post is %d,leak_stren is %d,bistable_stren is %d",(i,j,k,m,n,h,z)
                            dict_para[key] = classification_accuracy
                            
                    